In [10]:
import audb
import pandas as pd
import os
import glob
import re

In [2]:
datasets = audb.available()

targets = ["ravdess", "crema-d", "tess", "savee"]

datasets[datasets.index.isin(targets)].sort_values(by=["name", "version"], ascending=[True, False])

,backend,host,repository,version
name,,,,
crema-d,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.3.0
crema-d,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.2.0
crema-d,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.1.1
crema-d,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.1.0
crema-d,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.0.5
crema-d,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.0.3
crema-d,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.0.2
crema-d,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.0.1
ravdess,s3,s3.dualstack.eu-north-1.amazonaws.com,audb-public,1.1.3


In [3]:
class audb_dataset:
    def __init__(self, name: str, version: str | None = None):
        self.name = name
        self.version = version
        
crema_d = audb_dataset(name = "crema-d", version = "1.3.0")
ravdess = audb_dataset(name = "ravdess", version = "1.1.3")


#### CREMA-D

In [ ]:
crema_db = audb.load(crema_d.name,
                    version = crema_d.version,
                    format = "wav", 
                    sampling_rate= 16000,
                    mixdown = True,
                    pickle_tables= False,
                    cache_root="./"
)

In [4]:
crema_files = pd.read_csv('crema-d/1.3.0/fe182b91/db.files.csv')
crema_files

,file,speaker,corrupted
0,1001/1001_IEO_NEU_XX.wav,1001,False
1,1001/1001_IEO_HAP_LO.wav,1001,False
2,1001/1001_IEO_HAP_MD.wav,1001,False
3,1001/1001_IEO_HAP_HI.wav,1001,False
4,1001/1001_IEO_SAD_LO.wav,1001,False
...,...,...,...
7436,1091/1091_WSI_HAP_XX.wav,1091,False
7437,1091/1091_WSI_SAD_XX.wav,1091,False
7438,1091/1091_WSI_ANG_XX.wav,1091,False
7439,1091/1091_WSI_FEA_XX.wav,1091,False


In [5]:
crema_sentences = pd.read_csv('crema-d/1.3.0/fe182b91/db.sentence.csv')
crema_sentences

,file,sentence
0,1001/1001_IEO_NEU_XX.wav,IEO
1,1001/1001_IEO_HAP_LO.wav,IEO
2,1001/1001_IEO_HAP_MD.wav,IEO
3,1001/1001_IEO_HAP_HI.wav,IEO
4,1001/1001_IEO_SAD_LO.wav,IEO
...,...,...
7436,1091/1091_WSI_HAP_XX.wav,WSI
7437,1091/1091_WSI_SAD_XX.wav,WSI
7438,1091/1091_WSI_ANG_XX.wav,WSI
7439,1091/1091_WSI_FEA_XX.wav,WSI


In [19]:
crema_complete_df = crema_files.merge(crema_sentences, on = "file", how = "left")
crema_complete_df

,file,speaker,corrupted,sentence
0,1001/1001_IEO_NEU_XX.wav,1001,False,IEO
1,1001/1001_IEO_HAP_LO.wav,1001,False,IEO
2,1001/1001_IEO_HAP_MD.wav,1001,False,IEO
3,1001/1001_IEO_HAP_HI.wav,1001,False,IEO
4,1001/1001_IEO_SAD_LO.wav,1001,False,IEO
...,...,...,...,...
7436,1091/1091_WSI_HAP_XX.wav,1091,False,WSI
7437,1091/1091_WSI_SAD_XX.wav,1091,False,WSI
7438,1091/1091_WSI_ANG_XX.wav,1091,False,WSI
7439,1091/1091_WSI_FEA_XX.wav,1091,False,WSI


In [20]:
crema_complete_df = crema_complete_df[crema_complete_df["corrupted"] == False]
crema_complete_df.shape

(7440, 4)

In [21]:
EMOTION_LABELS = {
    "ANG": "anger",
    "DIS": "disgust",
    "FEA": "fear",
    "HAP": "happy/joy",
    "NEU": "neutral",
    "SAD": "sad"
}

SENTENCE_LABELS = {
    "IEO": "It's eleven o'clock",
    "TIE": "That is exactly what happened",
    "IOM": "I'm on my way to the meeting",
    "IWW": "I wonder what this is about",
    "TAI": "The airplane is almost full",
    "MTI": "Maybe tomorrow it will be cold",
    "IWL": "I would like a new alarm clock",
    "ITH": "I think I have a doctor's appointment",
    "DFA": "Don't forget a jacket",
    "ITS": "I think I've seen this before",
    "TSI": "The surface is slick",
    "WSI": "We'll stop in a couple of minutes"
}
        

In [9]:
example = "1091/1091_WSI_DIS_XX.wav"
example.replace("/", "_").replace(".", "_").split("_")[3]

def extract_emotion_code(file_path: str) -> str:
    return file_path.replace("/", "_").replace(".", "_").split("_")[3]

In [23]:
crema_complete_df["emotion_code"] = crema_complete_df["file"].apply(extract_emotion_code)
crema_complete_df["emotion"] = crema_complete_df["emotion_code"].map(EMOTION_LABELS)
crema_complete_df

,file,speaker,corrupted,sentence,emotion_code,emotion
0,1001/1001_IEO_NEU_XX.wav,1001,False,IEO,NEU,neutral
1,1001/1001_IEO_HAP_LO.wav,1001,False,IEO,HAP,happy/joy
2,1001/1001_IEO_HAP_MD.wav,1001,False,IEO,HAP,happy/joy
3,1001/1001_IEO_HAP_HI.wav,1001,False,IEO,HAP,happy/joy
4,1001/1001_IEO_SAD_LO.wav,1001,False,IEO,SAD,sad
...,...,...,...,...,...,...
7436,1091/1091_WSI_HAP_XX.wav,1091,False,WSI,HAP,happy/joy
7437,1091/1091_WSI_SAD_XX.wav,1091,False,WSI,SAD,sad
7438,1091/1091_WSI_ANG_XX.wav,1091,False,WSI,ANG,anger
7439,1091/1091_WSI_FEA_XX.wav,1091,False,WSI,FEA,fear


In [24]:
crema_complete_df.rename(columns={"sentence": "sentence_code"}, inplace=True)
crema_complete_df["sentence"] = crema_complete_df["sentence_code"].map(SENTENCE_LABELS)
crema_complete_df

,file,speaker,corrupted,sentence_code,emotion_code,emotion,sentence
0,1001/1001_IEO_NEU_XX.wav,1001,False,IEO,NEU,neutral,It's eleven o'clock
1,1001/1001_IEO_HAP_LO.wav,1001,False,IEO,HAP,happy/joy,It's eleven o'clock
2,1001/1001_IEO_HAP_MD.wav,1001,False,IEO,HAP,happy/joy,It's eleven o'clock
3,1001/1001_IEO_HAP_HI.wav,1001,False,IEO,HAP,happy/joy,It's eleven o'clock
4,1001/1001_IEO_SAD_LO.wav,1001,False,IEO,SAD,sad,It's eleven o'clock
...,...,...,...,...,...,...,...
7436,1091/1091_WSI_HAP_XX.wav,1091,False,WSI,HAP,happy/joy,We'll stop in a couple of minutes
7437,1091/1091_WSI_SAD_XX.wav,1091,False,WSI,SAD,sad,We'll stop in a couple of minutes
7438,1091/1091_WSI_ANG_XX.wav,1091,False,WSI,ANG,anger,We'll stop in a couple of minutes
7439,1091/1091_WSI_FEA_XX.wav,1091,False,WSI,FEA,fear,We'll stop in a couple of minutes


In [25]:
columns_order = ["file", "speaker", "corrupted", "sentence_code", "sentence", "emotion_code", "emotion"]
crema_complete_df = crema_complete_df[columns_order]
crema_complete_df

,file,speaker,corrupted,sentence_code,sentence,emotion_code,emotion
0,1001/1001_IEO_NEU_XX.wav,1001,False,IEO,It's eleven o'clock,NEU,neutral
1,1001/1001_IEO_HAP_LO.wav,1001,False,IEO,It's eleven o'clock,HAP,happy/joy
2,1001/1001_IEO_HAP_MD.wav,1001,False,IEO,It's eleven o'clock,HAP,happy/joy
3,1001/1001_IEO_HAP_HI.wav,1001,False,IEO,It's eleven o'clock,HAP,happy/joy
4,1001/1001_IEO_SAD_LO.wav,1001,False,IEO,It's eleven o'clock,SAD,sad
...,...,...,...,...,...,...,...
7436,1091/1091_WSI_HAP_XX.wav,1091,False,WSI,We'll stop in a couple of minutes,HAP,happy/joy
7437,1091/1091_WSI_SAD_XX.wav,1091,False,WSI,We'll stop in a couple of minutes,SAD,sad
7438,1091/1091_WSI_ANG_XX.wav,1091,False,WSI,We'll stop in a couple of minutes,ANG,anger
7439,1091/1091_WSI_FEA_XX.wav,1091,False,WSI,We'll stop in a couple of minutes,FEA,fear


In [27]:
crema_complete_df.to_csv("CREMA_D/crema_complete_files.csv", index = False)

#### RAVDESS

In [28]:
ravdess_db = audb.load(ravdess.name,
                    version = ravdess.version,
                    format = "wav", 
                    sampling_rate= 16000,
                    mixdown = True,
                    pickle_tables= False,
                    cache_root="./"
)

Get:   ravdess v1.1.3
Cache: /Users/alfred/Documents/SER/ravdess/1.1.3/fe182b91


In [31]:
from fastparquet import ParquetFile as fp
from pathlib import Path

In [38]:
ravdess_files = fp("RAVDESS/1.1.3/fe182b91/db.files.parquet").to_pandas()
ravdess_files

,file,speaker,duration,transcription,vocal channel
0,songs/actor_06/03-02-04-02-02-01-06.wav,06,4304291667,Dogs are sitting by the door,song
1,songs/actor_06/03-02-02-02-01-01-06.wav,06,5605583333,Kids are talking by the door,song
2,songs/actor_06/03-02-03-02-01-02-06.wav,06,4637979167,Kids are talking by the door,song
3,songs/actor_06/03-02-06-01-02-01-06.wav,06,4437770833,Dogs are sitting by the door,song
4,songs/actor_06/03-02-06-02-01-02-06.wav,06,4204208333,Kids are talking by the door,song
...,...,...,...,...,...
2447,speech/actor_08/03-01-06-01-01-01-08.wav,08,3970645833,Kids are talking by the door,speech
2448,speech/actor_08/03-01-01-01-01-02-08.wav,08,3636979167,Kids are talking by the door,speech
2449,speech/actor_08/03-01-03-01-01-01-08.wav,08,3503499999,Kids are talking by the door,speech
2450,speech/actor_08/03-01-05-02-01-02-08.wav,08,3770437500,Kids are talking by the door,speech


In [39]:
db_folder = Path("RAVDESS/1.1.3/fe182b91")
db_files = list(db_folder.glob("*.parquet"))

In [40]:
emotions_df = pd.DataFrame()
for path in db_files[1:]:
    
    try:
        df = fp(path).to_pandas()
        emotions_df = pd.concat([emotions_df, df], ignore_index=True)
        print(f"Concatenated {path.name} successfully.")

    except Exception as e:
        print(f"Failed to concatenate {path.name}: {e}")
        
emotions_df

Concatenated db.emotion.speech.dev.parquet successfully.
Concatenated db.emotion.song.dev.parquet successfully.
Concatenated db.emotion.speech.train.parquet successfully.
Concatenated db.emotion.song.train.parquet successfully.
Concatenated db.emotion.speech.test.parquet successfully.
Concatenated db.emotion.song.test.parquet successfully.


,file,emotion,emotional intensity
0,speech/actor_16/03-01-02-01-02-02-16.wav,calm,normal
1,speech/actor_16/03-01-05-01-02-02-16.wav,anger,normal
2,speech/actor_16/03-01-05-02-02-02-16.wav,anger,strong
3,speech/actor_16/03-01-02-01-01-02-16.wav,calm,normal
4,speech/actor_16/03-01-01-01-02-01-16.wav,neutral,normal
...,...,...,...
2447,songs/actor_22/03-02-04-01-02-01-22.wav,sadness,normal
2448,songs/actor_22/03-02-05-02-01-02-22.wav,anger,strong
2449,songs/actor_22/03-02-02-01-02-01-22.wav,calm,normal
2450,songs/actor_22/03-02-06-02-01-02-22.wav,fear,strong


In [42]:
ravdess_complete_df = ravdess_files.merge(emotions_df, on = "file", how = "left")
ravdess_complete_df.drop(columns=["emotional intensity"], inplace=True)
ravdess_complete_df

,file,speaker,duration,transcription,vocal channel,emotion
0,songs/actor_06/03-02-04-02-02-01-06.wav,06,4304291667,Dogs are sitting by the door,song,sadness
1,songs/actor_06/03-02-02-02-01-01-06.wav,06,5605583333,Kids are talking by the door,song,calm
2,songs/actor_06/03-02-03-02-01-02-06.wav,06,4637979167,Kids are talking by the door,song,happiness
3,songs/actor_06/03-02-06-01-02-01-06.wav,06,4437770833,Dogs are sitting by the door,song,fear
4,songs/actor_06/03-02-06-02-01-02-06.wav,06,4204208333,Kids are talking by the door,song,fear
...,...,...,...,...,...,...
2447,speech/actor_08/03-01-06-01-01-01-08.wav,08,3970645833,Kids are talking by the door,speech,fear
2448,speech/actor_08/03-01-01-01-01-02-08.wav,08,3636979167,Kids are talking by the door,speech,neutral
2449,speech/actor_08/03-01-03-01-01-01-08.wav,08,3503499999,Kids are talking by the door,speech,happiness
2450,speech/actor_08/03-01-05-02-01-02-08.wav,08,3770437500,Kids are talking by the door,speech,anger


In [44]:
if os.path.exists("RAVDESS/ravdess_complete_files.csv"):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    ravdess_complete_df.to_csv("RAVDESS/ravdess_complete_files.csv", index=False)
    print("File saved successfully.")


File saved successfully.


#### SAVEE

In [3]:
savee_root = "SAVEE/data"

file_paths = glob.glob(os.path.join(savee_root, "*.wav"))

savee_complete_df = pd.DataFrame(file_paths, columns=["file"])
print(savee_complete_df.shape)
savee_complete_df

(480, 1)


,file
0,SAVEE/data/JK_sa01.wav
1,SAVEE/data/JK_sa15.wav
2,SAVEE/data/DC_n13.wav
3,SAVEE/data/DC_su09.wav
4,SAVEE/data/DC_n07.wav
...,...
475,SAVEE/data/JE_a01.wav
476,SAVEE/data/JE_a15.wav
477,SAVEE/data/DC_n22.wav
478,SAVEE/data/DC_su04.wav


In [13]:
EMOTION_MAPPING_SAVEE = {
    'a': 'anger', 
    'd': 'disgust',
    'f': 'fear',
    'h': 'happy',
    'n': 'neutral',
    'sa': 'sadness',
    'su': 'surprise'
}

pattern_savee = r"(?P<speaker>[a-zA-Z]+)_(?P<emotion>[a-z]+)\d+\.wav"

def ext_speaker(file_path: str) -> str:
    base_name = os.path.basename(file_path)
    match = re.match(pattern_savee, base_name)
    if match:
        return match.group("speaker")
    return "unknown"

def ext_emotion(file_path: str) -> str:
    base_name = os.path.basename(file_path)
    match = re.match(pattern_savee, base_name)
    if match:
        return EMOTION_MAPPING_SAVEE.get(match.group("emotion"), "unknown")
    return "unknown"


In [16]:
savee_complete_df["emotion"] = savee_complete_df["file"].apply(ext_emotion)

savee_complete_df["speaker"] = savee_complete_df["file"].apply(ext_speaker)
savee_complete_df["source"] = "savee"
savee_complete_df

,file,emotion,speaker,source
0,SAVEE/data/JK_sa01.wav,sadness,JK,savee
1,SAVEE/data/JK_sa15.wav,sadness,JK,savee
2,SAVEE/data/DC_n13.wav,neutral,DC,savee
3,SAVEE/data/DC_su09.wav,surprise,DC,savee
4,SAVEE/data/DC_n07.wav,neutral,DC,savee
...,...,...,...,...
475,SAVEE/data/JE_a01.wav,anger,JE,savee
476,SAVEE/data/JE_a15.wav,anger,JE,savee
477,SAVEE/data/DC_n22.wav,neutral,DC,savee
478,SAVEE/data/DC_su04.wav,surprise,DC,savee


In [17]:
if os.path.exists("SAVEE/savee_complete_files.csv"  ):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    savee_complete_df.to_csv("SAVEE/savee_complete_files.csv", index = False)
    print("File saved successfully.")

File saved successfully.


#### TESS

In [20]:
tess_complete_df = pd.read_csv("TESS/MANIFEST.TXT", sep=" ", header=None, names=["file", "unused1", "unused2", "unused3"])
tess_complete_df


,file,unused1,unused2,unused3
0,OAF_shout_ps.wav,(audio/wav),102500,bytes.
1,YAF_pole_sad.wav,(audio/wav),114266,bytes.
2,YAF_check_disgust.wav,(audio/wav),131988,bytes.
3,YAF_shirt_angry.wav,(audio/wav),103620,bytes.
4,OAF_life_sad.wav,(audio/wav),125298,bytes.
...,...,...,...,...
2795,OAF_cab_neutral.wav,(audio/wav),93168,bytes.
2796,OAF_chain_disgust.wav,(audio/wav),114798,bytes.
2797,OAF_jar_neutral.wav,(audio/wav),100460,bytes.
2798,OAF_lot_angry.wav,(audio/wav),78798,bytes.


In [ ]:
tess_complete_df.drop(columns=["unused1", "unused2", "unused3"], inplace=True)

tess_root = os.path.join("TESS", "data")

tess_complete_df["file"] = tess_complete_df["file"].apply(lambda x: os.path.join(tess_root, x))
tess_complete_df

,file,unused1,unused2,unused3
0,TESS/data/OAF_shout_ps.wav,(audio/wav),102500,bytes.
1,TESS/data/YAF_pole_sad.wav,(audio/wav),114266,bytes.
2,TESS/data/YAF_check_disgust.wav,(audio/wav),131988,bytes.
3,TESS/data/YAF_shirt_angry.wav,(audio/wav),103620,bytes.
4,TESS/data/OAF_life_sad.wav,(audio/wav),125298,bytes.
...,...,...,...,...
2795,TESS/data/OAF_cab_neutral.wav,(audio/wav),93168,bytes.
2796,TESS/data/OAF_chain_disgust.wav,(audio/wav),114798,bytes.
2797,TESS/data/OAF_jar_neutral.wav,(audio/wav),100460,bytes.
2798,TESS/data/OAF_lot_angry.wav,(audio/wav),78798,bytes.


In [23]:
EMOTION_MAPPING_SAVEE = {
    'a': 'anger', 
    'd': 'disgust',
    'f': 'fear',
    'h': 'happy',
    'n': 'neutral',
    'sa': 'sadness',
    'su': 'surprise'
}

pattern_savee = r"(?P<speaker>[A-Z]+)_(?P<expression>[a-z]+)_(?P<emotion>[a-z]+)\.wav"

def ext_speaker(file_path: str) -> str:
    base_name = os.path.basename(file_path)
    match = re.match(pattern_savee, base_name)
    if match:
        return match.group("speaker")
    return "unknown"

def ext_emotion(file_path: str) -> str:
    base_name = os.path.basename(file_path)
    match = re.match(pattern_savee, base_name)
    if match:
        return match.group("emotion")
    return "unknown"



In [24]:
tess_complete_df["speaker"] = tess_complete_df["file"].apply(ext_speaker)
tess_complete_df["emotion"] = tess_complete_df["file"].apply(ext_emotion)
tess_complete_df

,file,speaker,emotion
0,TESS/data/OAF_shout_ps.wav,OAF,ps
1,TESS/data/YAF_pole_sad.wav,YAF,sad
2,TESS/data/YAF_check_disgust.wav,YAF,disgust
3,TESS/data/YAF_shirt_angry.wav,YAF,angry
4,TESS/data/OAF_life_sad.wav,OAF,sad
...,...,...,...
2795,TESS/data/OAF_cab_neutral.wav,OAF,neutral
2796,TESS/data/OAF_chain_disgust.wav,OAF,disgust
2797,TESS/data/OAF_jar_neutral.wav,OAF,neutral
2798,TESS/data/OAF_lot_angry.wav,OAF,angry


In [25]:
if os.path.exists("TESS/tess_complete_files.csv"  ):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    tess_complete_df.to_csv("TESS/tess_complete_files.csv", index = False)
    print("File saved successfully.")

File saved successfully.
